In [1]:
dataset1 = []

class DataPoint:
  def __init__(self, status, vars):
    self.isTrue = status
    self.title = vars[0]
    self.text = vars[1]
    self.subject = vars[2]
    self.data = vars[3]

  def print(self):
    print(self.isTrue + " - " + self.title + " - " + self.text)
    



In [2]:
import pandas as pd

df = pd.read_csv (r'Databases/Data1/Fake.csv')
ds1 = df[["title", "text"]]
ds1["status"] = False
df = pd.read_csv (r'Databases/Data1/True.csv')
ds2 = df[["title", "text"]]
ds2["status"] = True
dataset1 = pd.concat([ds1, ds2])
dataset1.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
print(dataset1.head)


c:\Users\jjfri\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


<bound method NDFrame.head of                                                    title  \
0       Donald Trump Sends Out Embarrassing New Year’...   
1       Drunk Bragging Trump Staffer Started Russian ...   
2       Sheriff David Clarke Becomes An Internet Joke...   
3       Trump Is So Obsessed He Even Has Obama’s Name...   
4       Pope Francis Just Called Out Donald Trump Dur...   
5       Racist Alabama Cops Brutalize Black Boy While...   
6       Fresh Off The Golf Course, Trump Lashes Out A...   
7       Trump Said Some INSANELY Racist Stuff Inside ...   
8       Former CIA Director Slams Trump Over UN Bully...   
9       WATCH: Brand-New Pro-Trump Ad Features So Muc...   
10      Papa John’s Founder Retires, Figures Out Raci...   
11      WATCH: Paul Ryan Just Told Us He Doesn’t Care...   
12      Bad News For Trump — Mitch McConnell Says No ...   
13      WATCH: Lindsey Graham Trashes Media For Portr...   
14      Heiress To Disney Empire Knows GOP Scammed Us...   
15      To

c:\Users\jjfri\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
LGVars = pd.DataFrame()
import numpy as np
import spacy as sp

# Load English tokenizer, tagger, parser and NER
nlp = sp.load("en_core_web_sm")

# Process whole documents
text = ("When Sebastian Thrun started working on self-driving cars at "
        "Google in 2007, few people outside of the company took him "
        "seriously. “I can tell you very senior CEOs of major American "
        "car companies would shake my hand and turn away because I wasn’t "
        "worth talking to,” said Thrun, in an interview with Recode earlier "
        "this week.")
doc = nlp(text)